# Look at debugging outputs

In [50]:
import os
import json
import numpy as np
from collections import defaultdict
import pprint

In [3]:
class args:
    pass

args.dout = 'exp_all/model:seq2seq_nl_baseline,name:full_0'

In [4]:
!ls -l exp_all/model:seq2seq_nl_baseline,name:full_0

total 0
-rwxrwxrwx 1 root root       448 Apr  8 17:17 best_seen.json
-rwxrwxrwx 1 root root 153981969 Apr  8 17:17 best_seen.pth
-rwxrwxrwx 1 root root       448 Apr  8 17:17 best_unseen.json
-rwxrwxrwx 1 root root 153981969 Apr  8 17:17 best_unseen.pth
-rwxrwxrwx 1 root root       856 Apr  8 17:17 config.json
-rwxrwxrwx 1 root root   1196706 Apr  8 17:17 events.out.tfevents.1586315808.1f9a25fd06f3
-rwxrwxrwx 1 root root 153981969 Apr  8 17:17 latest.pth
-rwxrwxrwx 1 root root         0 Apr  8 17:17 log.txt
-rwxrwxrwx 1 root root  47998821 Apr  8 17:17 train.debug_epoch_0.preds.json
-rwxrwxrwx 1 root root  47561778 Apr  8 17:17 train.debug_epoch_1.preds.json
-rwxrwxrwx 1 root root  47529048 Apr  8 17:17 train.debug_epoch_2.preds.json
-rwxrwxrwx 1 root root         0 Apr  8 17:17 train.debug_epoch_3.preds.json
-rwxrwxrwx 1 root root    588159 Apr  8 17:17 valid_seen.debug_epoch_0.preds.json
-rwxrwxrwx 1 root root    558041 Apr  8 17:17 valid_seen.debug_epoch_1.preds.json
-rwxrwxrwx 1 ro

In [59]:
num_epochs = 2

# Examine Outputs across Epochs

- across epochs
- random selection
- rank by CE, BLEU

### Select a random set of tasks to look at

In [37]:
np.random.seed(42)

path = os.path.join(args.dout, 'valid_seen.debug_epoch_0.preds.json')
with open(path, 'r') as f:
    debug_valid_seen = json.load(f)
chosen_valid_seen_task_ids = np.random.choice(list(debug_valid_seen.keys()), size=10, replace=False)

path = os.path.join(args.dout, 'valid_unseen.debug_epoch_0.preds.json')
with open(path, 'r') as f:
    debug_valid_unseen = json.load(f)
chosen_valid_unseen_task_ids = np.random.choice(list(debug_valid_unseen.keys()), size=10, replace=False)

def find_num_high_level_goals(chosen_task_ids, debug_dat):
    '''number of high level goals (complexity) of these chosen tasks'''
    for t in chosen_task_ids:
        print (t, '\t\t\t', len(debug_dat[t]['action_high']))
    
print("\nRandomly Selected for Valid Seen: (task, number of planner subgoals)")
find_num_high_level_goals(chosen_valid_seen_task_ids, debug_valid_seen)
print("\nRandomly Selected for Valid Unseen: (task, number of planner subgoals)")
find_num_high_level_goals(chosen_valid_unseen_task_ids, debug_valid_unseen)


Randomly Selected for Valid Seen: (task, number of planner subgoals)
trial_T20190909_150916_682139_0 			 5
trial_T20190908_192636_561572_0 			 8
trial_T20190907_013001_399963_0 			 12
trial_T20190907_074712_870488_0 			 8
trial_T20190908_102840_789300_0 			 9
trial_T20190909_031040_889814_0 			 5
trial_T20190908_140701_251653_0 			 7
trial_T20190909_055649_717880_0 			 9
trial_T20190907_151802_277016_0 			 13
trial_T20190909_032318_169393_0 			 5

Randomly Selected for Valid Unseen: (task, number of planner subgoals)
trial_T20190909_123918_983775_0 			 13
trial_T20190909_123732_489327_0 			 11
trial_T20190906_191445_723170_0 			 5
trial_T20190907_020329_050110_0 			 7
trial_T20190907_170955_824797_0 			 5
trial_T20190909_210238_431966_0 			 5
trial_T20190906_224843_443882_0 			 13
trial_T20190906_214631_761426_0 			 5
trial_T20190907_074524_006355_0 			 5
trial_T20190908_114656_768805_0 			 7


In [40]:
chosen_valid_seen_task_ids = [
'trial_T20190909_150916_682139_0', 
'trial_T20190908_192636_561572_0', 
'trial_T20190907_013001_399963_0', 
'trial_T20190907_074712_870488_0', 
'trial_T20190908_102840_789300_0', 
'trial_T20190909_031040_889814_0', 
'trial_T20190908_140701_251653_0', 
'trial_T20190909_055649_717880_0', 
'trial_T20190907_151802_277016_0', 
'trial_T20190909_032318_169393_0'
]

chosen_valid_unseen_task_ids = [
'trial_T20190909_123918_983775_0',
'trial_T20190909_123732_489327_0',
'trial_T20190906_191445_723170_0',
'trial_T20190907_020329_050110_0',
'trial_T20190907_170955_824797_0',
'trial_T20190909_210238_431966_0',
'trial_T20190906_224843_443882_0',
'trial_T20190906_214631_761426_0',
'trial_T20190907_074524_006355_0',
'trial_T20190908_114656_768805_0'
]

### Look at each task

In [ ]:
# TODO
# add all references
# add bleu score


In [57]:
sampled_task_ids = chosen_valid_seen_task_ids
num_epochs = 2
split = 'valid_seen'


def examine_sampled_tasks(sampled_task_ids, num_epochs, split):
    sampled_results = defaultdict(dict)

    for epoch in range(num_epochs):

        path = os.path.join(args.dout, '{}.debug_epoch_{}.preds.json'.format(split, epoch))
        with open(path, 'r') as f:
            debug_dat = json.load(f)

        for task_id in sampled_task_ids:
            if epoch == 0:
                sampled_results[task_id]['lang_goal'] = debug_dat[task_id]['lang_goal']
                sampled_results[task_id]['action_high'] = debug_dat[task_id]['action_high']
                sampled_results[task_id]['action_low'] = debug_dat[task_id]['action_low']
                sampled_results[task_id]['lang_instr'] = debug_dat[task_id]['lang_instr']
                sampled_results[task_id]['num_subgoals'] = len(debug_dat[task_id]['action_high'])
            sampled_results[task_id]['p_lang_instr_{}'.format(epoch)] = debug_dat[task_id]['p_lang_instr']

    print('SPLIT: {}\n\n\n'.format(split))
    for task_id in sampled_task_ids:
        print("TASK :", task_id)
        print("Number of subgoals :", sampled_results[task_id]['num_subgoals'])
        print ('---------------------------')
        print("Gold goal description:\n", sampled_results[task_id]['lang_goal'])
        print ('---------------------------')
        print("Gold Input high-level action description:\n", sampled_results[task_id]['action_high'])
        print ('---------------------------')
        print("Gold Input low-level action description:\n", sampled_results[task_id]['action_low'])
        print ('---------------------------')
        print("Gold low-level instruction description:\n", '.'.join(sampled_results[task_id]['lang_instr']))
        print ('---------------------------')
        for e in range(num_epochs):
            print("\nEpoch {} Prediction Output:\n".format(e), sampled_results[task_id]['p_lang_instr_{}'.format(e)])
        print ('--------------------------------------------------------------------------------------\n\n\n')    

In [60]:
examine_sampled_tasks(sampled_task_ids = chosen_valid_seen_task_ids, num_epochs = num_epochs, split = 'valid_seen')

SPLIT: valid_seen



TASK : trial_T20190909_150916_682139_0
Number of subgoals : 5
---------------------------
Gold goal description:
 Carry a purple pillow while turning on a lamp
---------------------------
Gold Input high-level action description:
 ['GotoLocation', 'PickupObject', 'GotoLocation', 'ToggleObject', 'NoOp']
---------------------------
Gold Input low-level action description:
 ['LookDown_15', 'RotateLeft_90', 'MoveAhead_25', 'MoveAhead_25', 'RotateLeft_90', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'RotateRight_90', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'RotateLeft_90', 'PickupObject', 'RotateLeft_90', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'RotateRight_90', 'MoveAhead_25', 'MoveAhead_25', 'RotateLeft_90', 'MoveAhead_25', 'LookDown_15', 'ToggleObjectOn']
---------------------------
Gold lo

In [61]:
examine_sampled_tasks(sampled_task_ids = chosen_valid_unseen_task_ids, num_epochs = num_epochs, split = 'valid_unseen')

SPLIT: valid_unseen



TASK : trial_T20190909_123918_983775_0
Number of subgoals : 13
---------------------------
Gold goal description:
 place a cooked potato slice in the fridge
---------------------------
Gold Input high-level action description:
 ['GotoLocation', 'PickupObject', 'GotoLocation', 'SliceObject', 'GotoLocation', 'PutObject', 'GotoLocation', 'PickupObject', 'GotoLocation', 'HeatObject', 'GotoLocation', 'PutObject', 'NoOp']
---------------------------
Gold Input low-level action description:
 ['LookDown_15', 'RotateRight_90', 'MoveAhead_25', 'MoveAhead_25', 'RotateRight_90', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'RotateRight_90', 'MoveAhead_25', 'MoveAhead_25', 'MoveAhead_25', 'RotateRight_90', 'LookDown_15', 'PickupObject', 